# Token Entropy Pattern Analysis

**Goal:** Find what's UNIQUE about token-level entropy that semantic entropy/consistency miss.

**Hypotheses:**
1. Entropy trajectory differs for factual vs hallucinated responses
2. First-token entropy predicts response quality
3. Entropy spikes on specific token types (names, numbers)

In [ ]:
!pip install -q transformers accelerate bitsandbytes

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)
model.eval()
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Model loaded!")

In [ ]:
def get_token_entropies(prompt, max_tokens=30):
    """Generate response and get entropy at each token position."""
    formatted = f"<s>[INST] {prompt} Answer briefly. [/INST]"
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    prompt_len = inputs.input_ids.shape[1]

    generated_ids = inputs.input_ids.clone()
    entropies = []
    tokens = []

    for _ in range(max_tokens):
        with torch.no_grad():
            outputs = model(generated_ids)
            logits = outputs.logits[0, -1]

        probs = F.softmax(logits.float(), dim=-1)
        entropy = -torch.sum(probs * torch.log(probs + 1e-10)).item()
        entropies.append(entropy)

        # Greedy decode
        next_token = torch.argmax(probs).unsqueeze(0).unsqueeze(0)
        tokens.append(tokenizer.decode(next_token[0]))
        generated_ids = torch.cat([generated_ids, next_token], dim=1)

        # Stop at EOS or period
        if next_token.item() == tokenizer.eos_token_id:
            break

    response = tokenizer.decode(generated_ids[0, prompt_len:], skip_special_tokens=True)
    return {
        "response": response,
        "entropies": entropies,
        "tokens": tokens,
        "first_entropy": entropies[0] if entropies else 0,
        "mean_entropy": np.mean(entropies),
        "max_entropy": np.max(entropies) if entropies else 0,
        "entropy_std": np.std(entropies) if entropies else 0,
        "entropy_trend": np.polyfit(range(len(entropies)), entropies, 1)[0] if len(entropies) > 1 else 0,
    }

## Experiment 1: Entropy Trajectories

In [ ]:
test_cases = [
    # Factual - should be confident throughout
    {"q": "What is the capital of France?", "type": "factual"},
    {"q": "Who wrote Romeo and Juliet?", "type": "factual"},
    {"q": "What is 2 + 2?", "type": "factual"},

    # Hallucination-prone - entropy pattern?
    {"q": "What did Einstein say about Bitcoin?", "type": "hallucination"},
    {"q": "What is the phone number of the Eiffel Tower?", "type": "hallucination"},
    {"q": "What was Aristotle's email address?", "type": "hallucination"},

    # Subjective - different pattern?
    {"q": "What is the best programming language?", "type": "subjective"},
    {"q": "What is the meaning of life?", "type": "subjective"},
]

results = []
for case in tqdm(test_cases):
    result = get_token_entropies(case["q"])
    result["question"] = case["q"]
    result["type"] = case["type"]
    results.append(result)

print("Done!")

In [ ]:
# Plot entropy trajectories by type
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, qtype in enumerate(["factual", "hallucination", "subjective"]):
    ax = axes[idx]
    type_results = [r for r in results if r["type"] == qtype]

    for r in type_results:
        ax.plot(r["entropies"], alpha=0.7, label=r["question"][:20])

    ax.set_title(f"{qtype.upper()} Questions")
    ax.set_xlabel("Token Position")
    ax.set_ylabel("Entropy")
    ax.legend(fontsize=8)
    ax.set_ylim(0, 3)

plt.tight_layout()
plt.show()

## Experiment 2: First-Token Entropy as Predictor

In [ ]:
import pandas as pd

df = pd.DataFrame(results)

print("="*60)
print("FIRST-TOKEN ENTROPY BY TYPE")
print("="*60)
summary = df.groupby("type").agg({
    "first_entropy": ["mean", "std"],
    "mean_entropy": ["mean", "std"],
    "entropy_trend": ["mean", "std"],
}).round(3)
print(summary)

print("\n" + "="*60)
print("KEY INSIGHT: First-token entropy")
print("="*60)
for qtype in ["factual", "hallucination", "subjective"]:
    type_data = df[df["type"] == qtype]
    print(f"{qtype:15s}: first={type_data['first_entropy'].mean():.3f}, mean={type_data['mean_entropy'].mean():.3f}")

## Experiment 3: Token-by-Token Analysis

In [ ]:
print("="*60)
print("TOKEN-LEVEL ENTROPY BREAKDOWN")
print("="*60)

for r in results:
    print(f"\n[{r['type'].upper()}] {r['question'][:40]}")
    print(f"Response: {r['response'][:60]}...")
    print("Token entropies:")

    # Show first 15 tokens with entropy
    for i, (tok, ent) in enumerate(zip(r['tokens'][:15], r['entropies'][:15])):
        bar = "█" * int(ent * 10)
        marker = "⚠️" if ent > 1.0 else "  "
        print(f"  {i:2d}: {tok:15s} {ent:.2f} {bar} {marker}")

## Experiment 4: Entropy Trend Analysis

In [ ]:
print("="*60)
print("ENTROPY TREND (slope) BY TYPE")
print("="*60)
print("Positive slope = entropy increases during response")
print("Negative slope = entropy decreases during response")
print()

for qtype in ["factual", "hallucination", "subjective"]:
    type_data = df[df["type"] == qtype]
    avg_trend = type_data['entropy_trend'].mean()
    direction = "↑ INCREASING" if avg_trend > 0.01 else ("↓ DECREASING" if avg_trend < -0.01 else "→ STABLE")
    print(f"{qtype:15s}: slope={avg_trend:+.4f} {direction}")

## Summary: What's Unique About Token Entropy?

In [ ]:
print("="*60)
print("POTENTIAL NOVEL FINDINGS")
print("="*60)

# Check if first-token entropy differs significantly
factual_first = df[df["type"] == "factual"]["first_entropy"].mean()
halluc_first = df[df["type"] == "hallucination"]["first_entropy"].mean()
first_gap = halluc_first - factual_first

print(f"\n1. FIRST-TOKEN ENTROPY:")
print(f"   Factual: {factual_first:.3f}")
print(f"   Hallucination: {halluc_first:.3f}")
print(f"   Gap: {first_gap:.3f}")
if abs(first_gap) > 0.1:
    print("   >>> NOVEL: First token alone can predict response type!")

# Check entropy trends
factual_trend = df[df["type"] == "factual"]["entropy_trend"].mean()
halluc_trend = df[df["type"] == "hallucination"]["entropy_trend"].mean()

print(f"\n2. ENTROPY TRAJECTORY:")
print(f"   Factual trend: {factual_trend:+.4f}")
print(f"   Hallucination trend: {halluc_trend:+.4f}")
if (factual_trend < 0 and halluc_trend > 0) or (factual_trend > 0 and halluc_trend < 0):
    print("   >>> NOVEL: Opposite trends between factual and hallucination!")

# Check entropy variance
factual_std = df[df["type"] == "factual"]["entropy_std"].mean()
halluc_std = df[df["type"] == "hallucination"]["entropy_std"].mean()

print(f"\n3. ENTROPY VARIANCE:")
print(f"   Factual std: {factual_std:.3f}")
print(f"   Hallucination std: {halluc_std:.3f}")
if abs(halluc_std - factual_std) > 0.1:
    print("   >>> NOVEL: Hallucinations have different entropy variance!")